# config

In [4]:
import os
#pwd_path = os.path.abspath(os.path.dirname(__file__))
pwd_path = os.path.abspath(os.path.dirname(os.path.abspath('')))
cged_train_paths = [
    os.path.join(pwd_path, '../data/cn/CGED/CGED18_HSK_TrainingSet.xml'),
    os.path.join(pwd_path, '../data/cn/CGED/CGED17_HSK_TrainingSet.xml'),
    os.path.join(pwd_path, '../data/cn/CGED/CGED16_HSK_TrainingSet.xml'),
    # os.path.join(pwd_path, '../data/cn/CGED/sample_HSK_TrainingSet.xml'),
]

sighan_train_path = os.path.join(pwd_path, '../data/cn/sighan_2015/train.tsv')

use_segment = True
segment_type = 'char'
dataset = 'sighan'

output_dir = os.path.join(pwd_path, 'output')
# Training data path.
train_path = os.path.join(output_dir, 'train.txt')
model_dir = os.path.join(output_dir, 'models')
model_path = os.path.join(model_dir, 'model.pth')
vocab_path = os.path.join(model_dir, 'vocab.txt')

# nets
word_embed_size = 200
hidden_size = 200
n_layers = 2
dropout=0.0

# train
epochs = 20
batch_size = 64
min_freq = 1
learning_rate = 1e-3

if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# preprocess

In [5]:
import os
import sys
from xml.dom import minidom

sys.path.append("../..")

from pycorrector.utils.tokenizer import segment

def parse_xml_file(path, use_segment, segment_type):
    print('Parse data from %s' % path)
    word_arr = []
    dom_tree = minidom.parse(path)
    docs = dom_tree.documentElement.getElementsByTagName('DOC')
    for doc in docs:
        # Input the text
        text = doc.getElementsByTagName('CORRECTION')[0]. \
            childNodes[0].data.strip()
        # Segment
        # strip() method removes any leading (spaces at the beginning) and trailing (spaces at the end) characters (space is the default leading characters (space is the default leading character to remove)
        word_seq = ' '.join(segment(text.strip(), cut_type=segment_type)) if use_segment else text.strip()
        word_arr.append(word_seq)
    return word_arr

def get_data_file(path, use_segment, segment_type):
    data_list = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip() 
            if line.startswith("#"):
                continue
            parts = line.split("\t")
            if len(parts) != 2:
                continue
            target = ' '.join(segment(parts[1].strip(), cut_type=segment_type)) if use_segment else parts[1].strip()
            data_list.append(target)
    return data_list

def save_corpus_data(data_list, data_path):
    dirname = os.path.dirname(data_path)
    os.makedirs(dirname, exist_ok=True)
    with open(data_path, 'w', encoding='utf-8') as f:
        count = 0
        for line in data_list:
            f.write(line + '\n')
            count += 1
        print("save line size:%d to %s" % (count, data_path))

    data_list = []
if dataset == 'sighan':
    data = get_data_file(sighan_train_path, use_segment, segment_type)
    data_list.extend(data)
else:
    for path in cged_train_paths:
        data_list.extend(parse_xml_file(path, use_segment, segment_type))
# save data
save_corpus_data(data_list, train_path)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/jarviszhang/Desktop/bioinformatics/pycorrector-master/pycorrector/../data/cn/sighan_2015/train.tsv'